In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import os

from common import OUTPUTPATH
from models import linear_pcc
import data

In [2]:
import warnings
warnings.filterwarnings('ignore') # Warnings all come from non-feature names which are useless.

In [3]:
# estimators here
STATE = np.random.RandomState(seed=1000)
linear_pcc.fit(data.x1, data.y)

In [4]:
num_importance = 100 # defult values show all variables
from sklearn.inspection import permutation_importance

def individual_importance_dataframe(est, X, y, scoring='neg_log_loss', num_importance=100000, n_repeats=30):
    """
    This algorithm use linear estimators and get the importance variables where p < 0.05
    To easy comparison, we set importance score: 1- p
    """
    r = permutation_importance(est, X, y, n_repeats=n_repeats, scoring=scoring)
    non_zero_indx = np.where(r.importances_mean > 0)
    indx = r.importances_mean[non_zero_indx].argsort()[::-1]
    important_variables = X.columns[non_zero_indx][indx]
    importance_scores = r.importances_mean[non_zero_indx][indx]
    coef = est.coef_[0][non_zero_indx][indx]
    df = pd.DataFrame({'variables': important_variables[:num_importance], 'coef': coef[:num_importance], 'importance': importance_scores[:num_importance]})
    return df

def linear_importance_dataframe(est, X, y, scoring = 'neg_log_loss', num_importance=100000, n_repeats=30):
    """
    Combine all morphologies plots together
    """
    data1 = pd.concat([X, y], axis=1)
    cnt = None
    for i in range(3):
        col_indx = -4 + i

        estimator, X, y = est[i], data1.iloc[:, :col_indx], data.y.iloc[:, i]
        temp_df = individual_importance_dataframe(est=estimator, X=X, y=y, scoring=scoring, num_importance=num_importance, n_repeats=n_repeats)
        if not cnt: 
            df = temp_df
            cnt = 1
        else:
            df = pd.concat([df, temp_df], axis=1)

    upper_columns = ['Sphere', 'Worm', 'Vesicle']
    lower_columns = ['variables', 'coef', 'importance']
    df.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
    return df

show all importance dataframe for Sphere, Worm, Vesicle

In [5]:
linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=100000)

Phase         Sphere                                Worm            \
Property   variables      coef    importance   variables      coef   
0         mw_tot_cre  0.000360  4.530841e+00  mw_tot_cre -0.000574   
1             dp_cre -0.037014  2.449948e+00  mv_tot_cre  0.042427   
2           apol_cna  0.312548  1.509042e+00      dp_cre  0.025709   
3         mv_tot_cna  0.119659  1.232580e+00  mw_tot_cna -0.000572   
4         mv_tot_cre -0.016181  1.098329e+00    apol_cna -0.127326   
5         mw_tot_cna -0.000747  6.492821e-01  mv_tot_cna  0.063264   
6             dp_cna  0.066770  3.841466e-01     psa_cre  0.150546   
7            psa_cna -0.044898  3.429419e-01     psa_cna  0.018629   
8            psa_cre -0.280656  2.545343e-01      mw_cna  0.008702   
9               conc -0.137321  8.263919e-02        conc  0.122434   
10            mw_cre  0.032254  1.943687e-02      mw_cre -0.049414   
11         clogp_cna  0.091527  8.107790e-03      sphere -0.060130   
12            mw_cna -0.001862  4.051063e-03        temp  0.009262   
13          apol_cre  0.033922  1.603349e-03   clogp_cna -0.037508   
14           charged  0.057333  6.722326e-04     charged -0.028757   
15         clogp_cre  0.021812  1.478862e-04    apol_cre  0.015980   
16              temp -0.002757  1.455823e-04      dp_cna -0.002180   
17        mass_ratio  0.026627  1.433032e-04          ph -0.009902   
18         vol_ratio  0.026373  1.415425e-04  mass_ratio -0.007020   
19              salt -0.001807  1.462761e-06   clogp_cre -0.012260   
20            mv_cna -0.000771  1.417016e-06   vol_ratio -0.006585   
21            mv_cre  0.000960  9.352485e-07      mv_cna  0.002739   
22               NaN       NaN           NaN        salt -0.001324   
23               NaN       NaN           NaN      mv_cre  0.000633   
24               NaN       NaN           NaN         NaN       NaN   

Phase                      Vesicle                          
Property    importance   variables      coef    importance  
0         7.791975e+00  mw_tot_cre -0.000205  2.408438e+00  
1         2.757101e+00  mv_tot_cre  0.013374  1.145326e+00  
2         1.306496e+00      dp_cre  0.015333  1.068779e+00  
3         5.879808e-01  mw_tot_cna -0.000156  5.955554e-02  
4         3.950927e-01     psa_cna  0.009302  3.392087e-02  
5         3.682914e-01      mw_cna -0.003252  1.271223e-02  
6         8.810649e-02        temp  0.013737  7.609308e-03  
7         8.518754e-02    apol_cna -0.012416  7.114367e-03  
8         8.445428e-02  mv_tot_cna  0.004310  4.491646e-03  
9         8.033545e-02        conc  0.011238  4.112410e-03  
10        2.897620e-02     psa_cre  0.006786  1.823525e-03  
11        2.129848e-03      mw_cre -0.006841  9.663109e-04  
12        2.034988e-03      dp_cna -0.002624  8.835263e-04  
13        1.837132e-03      sphere -0.003907  3.629037e-04  
14        6.544993e-04          ph -0.002704  1.470343e-04  
15        6.526845e-04   clogp_cna -0.000930  3.367805e-05  
16        4.536728e-04   clogp_cre -0.001075  2.631529e-05  
17        8.688559e-05    apol_cre  0.000386  1.354464e-05  
18        2.696516e-05        worm -0.000518  6.308156e-06  
19        2.635064e-05  mass_ratio -0.000460  4.919151e-06  
20        2.367080e-05   vol_ratio -0.000441  4.545344e-06  
21        6.660615e-06     charged -0.000402  2.074549e-06  
22        1.200038e-06      mv_cna  0.000114  1.100736e-06  
23        8.464549e-07        salt -0.000051  6.399945e-08  
24                 NaN      mv_cre  0.000025  4.786194e-08

In [7]:
top5_linear_dataframe = linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=5)
top5_linear_dataframe.to_csv(os.path.join(OUTPUTPATH, 'Top5_linear_dataframe.csv'))
top5_linear_dataframe


Phase         Sphere                             Worm                       \
Property   variables      coef importance   variables      coef importance   
0         mw_tot_cre  0.000360   4.537523  mw_tot_cre -0.000574   7.613551   
1             dp_cre -0.037014   2.450072  mv_tot_cre  0.042427   2.738487   
2           apol_cna  0.312548   1.496282      dp_cre  0.025709   1.295499   
3         mv_tot_cna  0.119659   1.217303  mw_tot_cna -0.000572   0.590799   
4         mv_tot_cre -0.016181   1.099023    apol_cna -0.127326   0.397351   

Phase        Vesicle                       
Property   variables      coef importance  
0         mw_tot_cre -0.000205   2.356357  
1         mv_tot_cre  0.013374   1.165152  
2             dp_cre  0.015333   1.083557  
3         mw_tot_cna -0.000156   0.062601  
4            psa_cna  0.009302   0.028074